# Titanic playground

Original Notebook: https://www.kaggle.com/code/marcpaulo/titanic-playground-for-new-kagglers-0-78

The goal of this Notebook is to present a simple way to approach the **Titanic Competition** found in the **Kaggle** platform. The best model found here will be used in our Streamlit App ```titanic_streamlit/main_app/streamlit_app.py```

The final solution (*Gradient Boosting Classifier*) presented here achieves a **0.78 score in the competition** (TOP 13~ on 07/09/2023). In the notebook ```titanic_streamlit/notebooks/training_grad_boost.ipynb``` you can find a short version of this *playground* notebook in which we train the Gradient Boosting Classifier directly.
       
The topics that we cover here are:
          
**1. Exploratory Data Analysis (EDA):**           
    - we'll explore and analyze column by column     
    - we'll see how we can detect missing values and outliers        
    - we'll visualize each feature (column) to check their distribution and structure         
    - we'll try to make an initial guess on the importance of each feature       
    - we'll use the *Pandas* and *Seaborn* libraries           
              
**2. Data Preprocessing:**          
    - we'll add a new feature (*feature extraction*)                            
    - we'll see different ways to deal with missing values                        
    - we'll use different modules from the *Sklearn* library to build clean and reusable *Pipelines* to process the data properly.
              
**3. Let's train some Models:**        
    - we'll test different basic classification algorithms            
    - we'll run many *GridSearch* with Cross-Validation for hyperparameter optimization      
    - we'll plot the *GridSearch* results to compare the performance of all the different settings
    
**4. Save the final Model:**                
    - train the best model architecture and save the trained parameters

# Let's get started!

In [ ]:
# Enter your Project Path in which the 'titanic_streamlit' folder is located:

notebook_config = {
    'your_project_path': '/home/marc/Escritorio/titanic_streamlit',  # TODO: fill this!!!
    
    'save_model': True,
    'model_file_name': 'trained_grad_boost.pkl',  # where the model is saved
    'run_sanity_check': True  # try to load the model afer it's saved
}


In [ ]:
import os
import pickle

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Read the train datasets using Pandas

In [ ]:
data_path = notebook_config['your_project_path'] + '/data/train.csv'
df_train = pd.read_csv(data_path)

In [ ]:
# lowercase all column names
df_train.columns = [c.lower() for c in df_train.columns]

First, let's take a look at the structure of the data.

In [ ]:
df_train.head()

In [ ]:
print(df_train.shape)

- **THE TASK AT HAND:**
    - a binary classification task. 11 features and 1 target ('Survived')
    - 891 training samples
            
            
- **COLUMNS DESCRIPTION**:   
    - **passengerid**: passenger identifier
    - **pclass**: ticket class {1: 1st, 2: 2nd, 3: 3rd}
    - **name**: passenger's name and honorific title
    - **sex**: 'male' or 'female'
    - **age**: in years
    - **sibsp**: number of siblings and spouses aboard
    - **parch**: number of parents and children aboard
    - **ticket**: ticket number
    - **fare**: passenger fare
    - **cabin**: cabin number
    - **embarked**: port of embarkation

# 1. Exploratory Data Analysis (EDA)

In this section, we are going to explore the data to summarize their main characteristics. We will use graphical tools to visualize the data and detect outliers, guess their distribution and try to gain a deeper understanding of the meaning of each feature (column).

In [ ]:
print(df_train.info())

**NULL VALUES:** (we will deal with them later)          
    - **age**: 20% is missing        
    - **cabin**: 77% is missing      
    - **embarked**: 2 samples are missing        

Now, let's visit each column to check if there are outliers, and their relationship with the target ('survived')

* **passengerid**: it has zero information, so we can safely remove it.

In [ ]:
df_train = df_train.drop(columns=['passengerid'])

Before, exploring the other columns, let's first create a small function that will help us to visualize each feature and analyze their distribution and importance. This function is intended to be used with categorical features or numerical features with a small set of unique values (e.g. a finite set of natural numbers).

In [ ]:
def eda_bars_plot(df_: pd.DataFrame, 
                  feature: str,
                  target: str = 'survived') -> None:
    """
    Crates two bar plots:
        - to check the distribution of 'feature'
        - to check the relationship between the 'target'
            and each category of 'feature'
    """

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
    df_val_counts = df_[feature].value_counts() / len(df_)
    df_val_counts = df_val_counts.reset_index().sort_values(by='index')
    sns.barplot(x='index', y=feature, data=df_val_counts, ax=ax1)
    ax1.set(xlabel=feature, 
            ylabel='% of each category', 
            title=feature + ': % value counts')

    df_group_by = df_.groupby(feature)[target].mean().reset_index()
    df_group_by = df_group_by.sort_values(by=feature)
    sns.barplot(x=feature, y=target, data=df_group_by, ax=ax2)
    ax2.set(xlabel=feature,
            ylabel=f'% {target}', 
            title=f"{feature}: prob of '{target}'")

* **pclass**: ticket class with values [1,2,3] = [Upper, Middle, Lower]        
In the plots we can see that more than 50% of the passengers bought lower class tickets. Unsurprisingly: the higher your class, the more likely you are to survive.

In [ ]:
eda_bars_plot(df_=df_train, feature='pclass')

* **sex**: We can see that only 40% of the passengers were females, but women had more than a 70% chance of surviving!. This means that we could build a classifier that predicts survival only for women (and death for men), and would achieve a decent accuracy. It could serve as a good first result (baseline).

In [ ]:
eda_bars_plot(df_=df_train, feature='sex')

* **age:** Almost 20% of the values are missing, but there don't seem to be any outliers since all values are within the normal range of ages.           
          
We definitely should NOT remove the rows with missing values since we would be losing a big portion of the data (too much information). Instead, we can *impute* the missing values. There are several ways to *impute* the missing values but, for simplicity, we will use the *median-imputer*. However, for more accurate results, other more advanced techniques may be used. In this case, for instance, we could use other categorical columns such as *'sex'*, *'pclass'* or the honorific title found in *'name'* to create a more specific *age-imputer* for each grup (category) of these features.

In [ ]:
ax_age = sns.histplot(x=df_train['age'])
_ = ax_age.set(title="'age' column histogram")

Looking at the plot below, it seems that the distribution of ages look quite similar for both the survivors and the non-survivors.          
                
**TODO:** In this section, we encourage you to explore the relationship between *'age'* and the other features. Maybe there is a combination of *'sex'+'age'* or *'pclass'+'sex'* groups which have something interesting to say in this classification problem (NOT TESTED!).

In [ ]:
sns.boxenplot(x=df_train['age'], y=df_train['survived'].astype('category'))

* **name**: it has zero information, so we can safely remove it.

**NOTE:** The honorific title (Mr, Mme, Lady, ...) can be extracted from the name and be used to create a new feature or to impute the missing *'age'* values.

In [ ]:
df_train = df_train.drop(columns=['name'])

* **sibsp**: the number of siblings and spouses aboard. It looks ok: neither outliers nor missing values. 
        
In the right-hand plot we can see that the you were more likely to survive if you had at list one sibling or a spouse abroad, but not too many of them.

In [ ]:
eda_bars_plot(df_=df_train, feature='sibsp')

* **parch**: number of parents and children aboard. In the next section we will sum this feature and *'sibsp'* to create a new feature *'num_relatives'* (*feature extraction*)

In [ ]:
eda_bars_plot(df_=df_train, feature='parch')

- **Fare:** There is an extremely high value around 500. Since it is very distant from the rest of the values, we will regard it as an outlier. We will set the maximum value to 300 (so we will cut the greater values to this new maximum).

In [ ]:
sns.histplot(x=df_train['fare'])
plt.axvline(300, color='red', alpha=0.4, linestyle='--')

In [ ]:
# there are some outliers, lets cut the maximum value to be 300
df_train.loc[df_train['fare'] > 300, 'fare'] = 300

As we can see: the higher the fare, the more likely you are to survive.

**TODO:** Our intuition tells us that the features *pclass* and *fare* might have some sort of correlation. Can you come up with a great plot to check if we are wrong? 

In [ ]:
sns.boxenplot(x=df_train['fare'], y=df_train['survived'].astype('category'))

* **cabin**: let's remove this columns since there are too many null values (~80% of them are missing)

In [ ]:
df_train = df_train.drop(columns=['cabin'])

* **embarked**: It turns out that the port of embarkation is more important than what you might expect. Let's see if our models are able to capture the importance of this feature!

In [ ]:
eda_bars_plot(df_=df_train, feature='embarked')

* **ticket**: we will assume that it has zero information, so we will remove it

In [ ]:
df_train = df_train.drop(columns=['ticket'])

# 2. Data Preprocessing

In this section, we will see different ways to deal with missing values depending on the type of features. We will also create a sequence of ordered *Sklearn Pipelines* to make our code reusable and avoid some typical problems such as *data leakage*. For the numerical features, we will apply a *MinMaxScaler* to map the values into the [0,1] interval. For the categorical features, we will apply a *OneHotEncoder*.

* **Feature extraction:** let's create a new feature from the already-existing ones. The number of relatives (family size) might be useful here.

In [ ]:
## a new feature: num_relatives = sibsp + parch
df_train['num_relatives'] = df_train['sibsp'] + df_train['parch']

It tourns out that 60% of the passengers were alone (at least with no family aboard). Small families (1,2,3 people) seem to be more likely to survive (more than 50% chance)

In [ ]:
eda_bars_plot(df_=df_train, feature='num_relatives')

Let's separate the training features (*df_train*) from the target (*y_train*)

In [ ]:
y_train = df_train['survived'].values  # [0,1]
df_train = df_train.drop(columns=['survived'])

In [ ]:
df_train.head()

Now, we will implement the *Preprocessing Pipeline* using *Sklearn*. Remember that columns *'age', 'fare', and 'embarked'* have at least one missing value. The transformations we will apply are:            
    - **pclass**: OneHotEncoder  (values [1,2,3])             
    - **sex**: OneHotEncoder  (values [0,1])            
    - **age**: Median-Imputer + MinMaxScaler            
    - **sibsp**: MinMaxScaler           
    - **parch**: MinMaxScaler           
    - **fare**: MinMaxScaler           
    - **embarked**: MostFrequentImputer + OneHotEncoder  (values ['S','C','Q'])        
    - **num_relatives**: MinMaxScaler       

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# First, create three small pipelines to stack more than one Transformer
# for a specific feature. Those features that only require one 
# transformation are handled by the final 'preprocessor' here below.
age_pipe = Pipeline(steps=[
    ('age_imp', SimpleImputer(strategy='median')),
    ('age_scale', MinMaxScaler())
])
embarked_pipe = Pipeline(steps=[
    ('embarked_imp', SimpleImputer(strategy='most_frequent')),
    ('embarked_onehot', OneHotEncoder(drop=None))
])

# Let's create the final 'preprocessor'
preprocessor = ColumnTransformer(
    transformers=[
        ('age_pipe', age_pipe, ['age']),
        ('embarked_pipe', embarked_pipe, ['embarked']),
        ('minmax_scaler', MinMaxScaler(), ['fare', 'sibsp', 'parch', 'num_relatives']),
        ('pclass_onehot', OneHotEncoder(drop=None), ['pclass']),
        ('sex_onehot', OneHotEncoder(drop='first'), ['sex'])
    ]
)

# 3. Let's train some Models

Once the data is ready, we can move on to the **'Modelling'** section. We will test different basic Machine Learning models and perform **Grid-Search Cross-Validation** to optimize the most interesting hyperparameters. In order to visualize the CV results, we will plot them using different techniques depending on the number of hyperparameters to optimize.

In [ ]:
from sklearn.model_selection import GridSearchCV

Let's start with one of the simplest Classification models: **LogisticRegression**. We will run a Grid Search to optimize the regularization parameter **'C'**. Then, we will plot the results to visualize how the accuracy depends on the 'C' value, and select the best model (the one achieving the best validation accuracy in the cross-validation).
        
In the plot below, we can see that the best result of **~79% accuracy** is achieved with **'C'= 1**

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('log_reg', LogisticRegression())
])
log_reg_params = {'log_reg__C': [1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3]}

log_reg_cv = GridSearchCV(estimator=log_reg, 
                          param_grid=log_reg_params,
                          scoring='accuracy',
                          n_jobs=-1,
                          cv=5,
                          verbose=1,
                          return_train_score=True
                         )
log_reg_cv.fit(df_train, y_train)

In [ ]:
print('best params:', log_reg_cv.best_params_)
print('best validation score:', log_reg_cv.best_score_)

In [ ]:
plt.plot(
    range(len(log_reg_params['log_reg__C'])), 
    log_reg_cv.cv_results_['mean_train_score'],
    label='train',
    color='blue'
)
plt.plot(
    range(len(log_reg_params['log_reg__C'])),
    log_reg_cv.cv_results_['mean_test_score'],
    label='test',
    color='green'
)
plt.xticks(ticks=range(len(log_reg_params['log_reg__C'])),
           labels=log_reg_params['log_reg__C'])
plt.legend()
plt.axvline(x=3, linestyle='--', color='red', alpha=0.3)
plt.xlabel("'C' vals")
plt.ylabel('accuracy')
plt.title("LogisticRegression GridSearchCV for 'C' param")
plt.show()

Now, we will test the **Support Vector Classifier** with the **Radial Kernel**. Here the goal is to optimize the regularization parameter **'C'**, and the kernel coefficient **'gamma'**. Since we have two hyperparameters to optimize, here we will display a *heat map* to show the results on how the accuracy depends on 'C' and 'gamma'.
        
In the heat map below, we can see that the best result of **~81% accuracy** is achieved with **'C'= 1** and **'gamma' = 1**

In [ ]:
from sklearn.svm import SVC

svc = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', SVC(kernel='rbf'))
])
svc_params = {
    'svc__C': [1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3],
    'svc__gamma': [1e-2, 1e-1, 1, 10, 100]
}
              
svc_cv = GridSearchCV(
    estimator=svc, 
    param_grid=svc_params,
    scoring='accuracy',
    n_jobs=-1,
    cv=5,
    verbose=1,
    return_train_score=False
)
svc_cv.fit(df_train, y_train)

In [ ]:
print('best params:', svc_cv.best_params_)
print('best validation score:', svc_cv.best_score_)

In [ ]:
ss = svc_cv.cv_results_['mean_test_score'].reshape(
    (len(svc_params['svc__C']), len(svc_params['svc__gamma']))
)
ax = sns.heatmap(ss, vmin=0.6, vmax=0.85, linewidth=0.5, annot=True)
_ = ax.set(
    xlabel='gamma', 
    ylabel='C', 
    title='SVC Validation Accuracy GridSearchCV',
    xticklabels=svc_params['svc__gamma'],
    yticklabels=svc_params['svc__C']
)

Now, we will try the **Random Forest Classifier**. The hyperparameters of this model are the number of decision trees **'n_estimators'**, and the maximum depth of each tree **'max_depth'**. Since we have two hyperparameters to optimize, here we will display a *heat map* to show the results on how the accuracy depends on 'n_estimators' and 'max_depth'. In general, the *Random Forest* algorithm usually works well with the default values for the rest of hyperparameters.

Note that the Random Forest can learn from features in different scales because they are processed separately. Although we could simplify our *'preprocessor'* by removing the *MinMaxScalers*, we will keep it the same for all models.
        
In the heat map below, we can see that the best result of **~83% accuracy** is achieved with **'n_estimators'= 100** and **'max_depth' = 8**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rf', RandomForestClassifier(random_state=12345))
])
rf_params = {
    'rf__n_estimators': [50, 100, 200, 400, 600, 800, 1000],
    'rf__max_depth': [1, 4, 8, 10]
}

rf_cv = GridSearchCV(
    estimator=rf, 
    param_grid=rf_params,
    scoring='accuracy',
    n_jobs=-1,                  
    cv=5,          
    verbose=1,
    return_train_score=True
)
rf_cv.fit(df_train, y_train)

In [ ]:
print('best RandomForest params:', rf_cv.best_params_)
print('best RandomForest cv accuracy:', rf_cv.best_score_)

In [ ]:
ss = rf_cv.cv_results_['mean_test_score'].reshape(
    (len(rf_params['rf__max_depth']), len(rf_params['rf__n_estimators']))
)
ax = sns.heatmap(ss, vmin=0.7, vmax=0.85, linewidth=0.5, annot=True)
_ = ax.set(
    xlabel='n_estimators', 
    ylabel='max_depth', 
    title='RandomForest Accuracy GridSearchCV',
    xticklabels=rf_params['rf__n_estimators'],
    yticklabels=rf_params['rf__max_depth']
)

The next algorithm is the **Gradient Boosting Classifier**. This algorithm performs a gradient optimization on a *loss function*. With this kind of *additive* model (such as the *Random Forest*), the default hyperparameter values tend to perform well enough. Here we will optimize the **'n_estimators'** (number of decision trees), and the **'learning_rate'**. As we did with other models, we will show the results in a *heat map*.

In the heat map below, we can see that the best result of **~83% accuracy** is achieved with **'n_estimators'= 400** and **'learning_rate' = 0.1**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

grad_boost = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('grad_boost', GradientBoostingClassifier(random_state=12345))
])
grad_boost_params = {
    'grad_boost__n_estimators': [50, 100, 200, 400, 600, 800, 1000],
    'grad_boost__learning_rate': [1e-4, 1e-3, 1e-2, 1e-1]}

grad_boost_cv = GridSearchCV(
    estimator=grad_boost, 
    param_grid=grad_boost_params,
    scoring='accuracy',
    n_jobs=-1,
    cv=5,
    verbose=1,
    return_train_score=False
)
grad_boost_cv.fit(df_train, y_train)

In [ ]:
print('best GradientBoosting params:', grad_boost_cv.best_params_)
print('best GradientBoosting cv accuracy:', grad_boost_cv.best_score_)

In [ ]:
ss = grad_boost_cv.cv_results_['mean_test_score'].reshape(
    (len(grad_boost_params['grad_boost__learning_rate']), 
     len(grad_boost_params['grad_boost__n_estimators']))
)
ax = sns.heatmap(ss, vmin=0.6, vmax=0.85, linewidth=0.5, annot=True)
_ = ax.set(
    xlabel='n_estimators', 
    ylabel='learning_rate', 
    title='GradientBoosting Accuracy GridSearchCV',
    xticklabels=grad_boost_params['grad_boost__n_estimators'],
    yticklabels=grad_boost_params['grad_boost__learning_rate']
)

The last algorithm is the **K-Nearest Neighbors Classifier**. We will use the *Euclidean* distance (the metric by default) and we will play with the **'n_neighbors** hyperparam.

In the plot below, we can see that the best result of **~80% accuracy** is achieved with **'n_neighbors'= 32**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', KNeighborsClassifier())
])
knn_params = {
    'knn__n_neighbors': [2, 4, 8, 16, 32, 64, 92, 128]
}

knn_cv = GridSearchCV(estimator=knn, 
                     param_grid=knn_params,
                     scoring='accuracy',
                      n_jobs=-1,
                      cv=5,
                      verbose=1,
                      return_train_score=True
                     )
knn_cv.fit(df_train, y_train)

In [ ]:
print('best KNearestNeighbors params:', knn_cv.best_params_)
print('best KNearestNeighbors cv accuracy:', knn_cv.best_score_)

In [ ]:
plt.plot(
    range(len(knn_params['knn__n_neighbors'])), 
    knn_cv.cv_results_['mean_train_score'],
    label='train',
    color='blue'
)
plt.plot(
    range(len(knn_params['knn__n_neighbors'])),
    knn_cv.cv_results_['mean_test_score'],
    label='test',
    color='green'
)
plt.xticks(ticks=range(len(knn_params['knn__n_neighbors'])),
           labels=knn_params['knn__n_neighbors'])
plt.legend()
plt.axvline(x=4, linestyle='--', color='red', alpha=0.3)
plt.xlabel("'n_neighbors' vals")
plt.ylabel('accuracy')
plt.title("KNearestNeighbour GridSearchCV for 'n_neighbors' param")
plt.show()

**TODO:** Try other classifiers! Some of the most promising algorithms are:   
    * **CatBoost**  ---  [link](https://catboost.ai/)                
    * **XGBoost**  ---  [link](https://xgboost.readthedocs.io/en/stable/python/python_api.html)            
    * **LightGBM**  ---  [link](https://lightgbm.readthedocs.io/en/stable/Python-API.html)
       
You can also explore the **Deep Learning** univers and build a Feed-Forward Neural Network using [Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html), [TensorFlow](https://www.tensorflow.org/api_docs/python/tf), or [PyTorch](https://pytorch.org/docs/stable/index.html)

# 4. Save the final model

As we have seen in the previous section, it turns out that the **Gradient Boosting** with 400 estimators and a learning rate of 0.1 is the best model we came up with (the best cross-validation accuracy). However, looking at the heatmap of the Cross-Validation results, we can see that with 100 estimators we also get an ~83% validation accuracy, with a 75% of reduction in the number of estimators. It means that both values of 'n_estimator' perform similarly.

For the sake of simplicity, let's choose the *'small'* model of **100 Decision Trees** and a **learning rate of 0.1**. A bigger model is more likely to overfit. Usually in Machine Learning: the simpler, the better.

Now, we will run a *10-Fold Cross-Validation* again to double-check the performance of the final model (expected to be an **83% accuracy**).
           
The last step is to train the final model using the whole dataset and save the trained parameters.

In [ ]:
from sklearn.model_selection import cross_val_score

best_grad_boost = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('grad_boost', GradientBoostingClassifier(
        n_estimators=100, 
        learning_rate=0.1, 
        random_state=12345
    ))
])

best_grad_boost_acc = cross_val_score(
    estimator=best_grad_boost,
    X=df_train,
    y=y_train,
    scoring='accuracy',
    cv=10,
    n_jobs=-1
)

print('best GradientBoosting acc (mean) =', round(np.mean(best_grad_boost_acc), 2))
print('best GradientBoosting acc (std)  =', round(np.std(best_grad_boost_acc), 2))

In [ ]:
grad_boost = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('grad_boost', GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        random_state=12345
    ))
])

grad_boost.fit(df_train, y_train)

In [ ]:
# Save the trained model

save_model_path = (
        notebook_config['your_project_path'] + 
        '/models/' + 
        notebook_config['model_file_name']
    )

if notebook_config['save_model']:
    
    
    with open(save_model_path, 'wb') as out_file:
        pickle.dump(grad_boost, out_file)
    print(f"Grad Boost model saved in:\n'{save_model_path}'")

else:

    print('According to the notebook_config, the model is NOT saved')

In [ ]:
# SANITY CHECK: Load the model

if notebook_config['run_sanity_check']:
    with open(save_model_path, 'rb') as in_file:
        loaded_model = pickle.load(in_file)

    print(f"Grad Boost model loaded from:\n'{save_model_path}'")
    print('train score:', loaded_model.score(df_train, y_train))

else:
    
    print('According to the notebook_config, do NOT run Sanity Check')